# NSGA-III Hyperparameter Optimization for Mammography Classification

Multi-objective hyperparameter optimization for deep learning models in breast cancer detection.

**Objectives:**
- Maximize: Sensitivity, Specificity, AUC
- Minimize: Model Size, Inference Time

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/dtobi59/-nsga3-mammography.git
%cd -nsga3-mammography

In [ ]:
# Install required packages
!pip install -q -r requirements.txt

## 4. Configure Dataset Path

Update the path below to point to your dataset on Google Drive.

**Supported datasets:**
- `vindr` - VinDr-Mammo
- `inbreast` - INbreast

**Expected structure:**

**VinDr-Mammo:**
```
vindr-mammo/
├── images/
│   └── {study_id}/
│       └── {image_id}.dicom
└── metadata/
    └── breast-level_annotations.csv
```

**INbreast:**
```
inbreast/
└── INbreast Release 1.0/
    ├── AllDICOMs/
    │   └── {id}_{hash}_MG_{L/R}_{CC/MLO}_ANON.dcm
    └── INbreast.csv
```

In [ ]:
# Configure your dataset
DATASET_NAME = "vindr"  # or "inbreast"
DATA_ROOT = "/content/drive/MyDrive/vindr-mammo"  # Update this path!
OUTPUT_DIR = "/content/drive/MyDrive/nsga3_outputs"  # Where to save results

## 5. Load Dataset

In [ ]:
from dataset import prepare_dataset

print(f"Loading {DATASET_NAME} dataset from {DATA_ROOT}...")

train_paths, train_labels, val_paths, val_labels = prepare_dataset(
    dataset_name=DATASET_NAME,
    data_root=DATA_ROOT
)

print(f"\nDataset loaded successfully!")
print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Train labels distribution: {sum(train_labels)} malignant, {len(train_labels) - sum(train_labels)} benign")
print(f"Val labels distribution: {sum(val_labels)} malignant, {len(val_labels) - sum(val_labels)} benign")

## 6. Configure Optimization

Adjust these parameters based on your computational budget:
- **pop_size**: Population size (more = better exploration, longer runtime)
- **n_generations**: Number of generations (more = better convergence)
- **epochs**: Training epochs per evaluation (reduce for faster testing)

In [ ]:
from config import ExperimentConfig

# Load default config
config = ExperimentConfig()

# Customize NSGA-III parameters
config.nsga3.pop_size = 20        # Recommended: 20-50 for quick runs, 100+ for thorough search
config.nsga3.n_generations = 10   # Recommended: 5-10 for testing, 20-50 for production

# Training epochs per evaluation (reduce for faster testing)
TRAINING_EPOCHS = 5  # Recommended: 5 for testing, 10-20 for production

print(f"Optimization Configuration:")
print(f"  Population size: {config.nsga3.pop_size}")
print(f"  Generations: {config.nsga3.n_generations}")
print(f"  Epochs per evaluation: {TRAINING_EPOCHS}")
print(f"  Total evaluations: ~{config.nsga3.pop_size * config.nsga3.n_generations}")
print(f"\nSearching hyperparameters:")
for param, values in config.hyperparameter_space.items():
    print(f"  {param}: {values}")

## 7. Create Evaluation Function

In [ ]:
from training import full_evaluation

def make_eval_fn(tp, tl, vp, vl, epochs=5):
    """
    Creates an evaluation function for the optimizer.
    
    Args:
        tp: Training paths
        tl: Training labels
        vp: Validation paths
        vl: Validation labels
        epochs: Number of training epochs
    
    Returns:
        Evaluation function that takes hyperparameter config and returns objectives
    """
    def eval_fn(hp_config):
        hp_config = hp_config.copy()
        hp_config['epochs'] = epochs
        return full_evaluation(
            hp_config, 
            tp, tl, vp, vl, 
            device='cuda',  # Use GPU
            verbose=True
        )
    return eval_fn

eval_function = make_eval_fn(
    train_paths, train_labels, 
    val_paths, val_labels,
    epochs=TRAINING_EPOCHS
)

print("Evaluation function created successfully!")

## 8. Run Optimization

This will take some time depending on your configuration.

**Estimated runtime:**
- Small (pop=20, gen=5, epochs=5): ~30-60 minutes
- Medium (pop=50, gen=10, epochs=10): ~2-4 hours
- Large (pop=100, gen=20, epochs=20): ~8-12 hours

In [ ]:
from optimization import run_optimization
import time

print("Starting optimization...\n")
start_time = time.time()

results = run_optimization(
    hp_space=config.hyperparameter_space,
    nsga_config=config.nsga3,
    eval_function=eval_function,
    output_dir=OUTPUT_DIR
)

elapsed_time = time.time() - start_time
print(f"\n{'='*70}")
print(f"Optimization completed in {elapsed_time/3600:.2f} hours")
print(f"{'='*70}")

## 9. View Results

In [ ]:
# Display Pareto front solutions
print(f"\nFound {len(results['pareto_configs'])} Pareto-optimal solutions\n")
print("="*100)

for i, (cfg, obj) in enumerate(zip(results['pareto_configs'], results['pareto_F'])):
    print(f"\nSolution {i+1}:")
    print("-" * 100)
    
    # Architecture details
    print(f"  Architecture:")
    print(f"    Backbone: {cfg['backbone']}")
    print(f"    Unfreeze: {cfg['unfreeze_strategy']}")
    print(f"    Dropout: {cfg['dropout_rate']:.2f}")
    print(f"    FC Hidden: {cfg['fc_hidden_size']}")
    
    # Training details
    print(f"  Training:")
    print(f"    Optimizer: {cfg['optimizer']}")
    print(f"    Learning Rate: {cfg['learning_rate']:.6f}")
    print(f"    Batch Size: {cfg['batch_size']}")
    print(f"    Loss: {cfg['loss_function']}")
    if cfg['loss_function'] == 'focal':
        print(f"    Focal Gamma: {cfg.get('focal_gamma', 2.0):.2f}")
    
    # Augmentation
    print(f"  Augmentation:")
    print(f"    Horizontal Flip: {cfg['horizontal_flip']}")
    print(f"    Rotation: {cfg['rotation_range']:.1f}°")
    print(f"    Mixup: {cfg['use_mixup']}")
    
    # Performance metrics
    print(f"  Performance:")
    print(f"    Sensitivity: {obj[0]:.4f}")
    print(f"    Specificity: {obj[1]:.4f}")
    print(f"    AUC: {obj[2]:.4f}")
    print(f"    Model Size: {obj[3]:.2f}M parameters")
    print(f"    Inference Time: {obj[4]:.2f}ms")

## 10. Visualize Pareto Front (Optional)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract objectives
pareto_F = np.array(results['pareto_F'])

# Create visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Pareto Front - Trade-off Analysis', fontsize=16, fontweight='bold')

# Sensitivity vs Specificity
axes[0, 0].scatter(pareto_F[:, 0], pareto_F[:, 1], c='blue', s=100, alpha=0.6)
axes[0, 0].set_xlabel('Sensitivity', fontsize=12)
axes[0, 0].set_ylabel('Specificity', fontsize=12)
axes[0, 0].set_title('Sensitivity vs Specificity')
axes[0, 0].grid(True, alpha=0.3)

# AUC vs Model Size
axes[0, 1].scatter(pareto_F[:, 2], pareto_F[:, 3], c='green', s=100, alpha=0.6)
axes[0, 1].set_xlabel('AUC', fontsize=12)
axes[0, 1].set_ylabel('Model Size (M params)', fontsize=12)
axes[0, 1].set_title('AUC vs Model Size')
axes[0, 1].grid(True, alpha=0.3)

# AUC vs Inference Time
axes[0, 2].scatter(pareto_F[:, 2], pareto_F[:, 4], c='red', s=100, alpha=0.6)
axes[0, 2].set_xlabel('AUC', fontsize=12)
axes[0, 2].set_ylabel('Inference Time (ms)', fontsize=12)
axes[0, 2].set_title('AUC vs Inference Time')
axes[0, 2].grid(True, alpha=0.3)

# Sensitivity vs Model Size
axes[1, 0].scatter(pareto_F[:, 0], pareto_F[:, 3], c='purple', s=100, alpha=0.6)
axes[1, 0].set_xlabel('Sensitivity', fontsize=12)
axes[1, 0].set_ylabel('Model Size (M params)', fontsize=12)
axes[1, 0].set_title('Sensitivity vs Model Size')
axes[1, 0].grid(True, alpha=0.3)

# Model Size vs Inference Time
axes[1, 1].scatter(pareto_F[:, 3], pareto_F[:, 4], c='orange', s=100, alpha=0.6)
axes[1, 1].set_xlabel('Model Size (M params)', fontsize=12)
axes[1, 1].set_ylabel('Inference Time (ms)', fontsize=12)
axes[1, 1].set_title('Model Size vs Inference Time')
axes[1, 1].grid(True, alpha=0.3)

# Overall metrics distribution
metrics = ['Sensitivity', 'Specificity', 'AUC', 'Size (M)', 'Time (ms)']
avg_values = pareto_F.mean(axis=0)
axes[1, 2].bar(metrics, avg_values, color=['blue', 'green', 'red', 'purple', 'orange'], alpha=0.6)
axes[1, 2].set_ylabel('Average Value', fontsize=12)
axes[1, 2].set_title('Average Objective Values')
axes[1, 2].tick_params(axis='x', rotation=45)
axes[1, 2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/pareto_front_visualization.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nVisualization saved to {OUTPUT_DIR}/pareto_front_visualization.png")

## 11. Save Best Configuration for Deployment

In [ ]:
import json

# Find solution with best AUC
best_auc_idx = np.argmax(pareto_F[:, 2])
best_auc_config = results['pareto_configs'][best_auc_idx]
best_auc_objectives = pareto_F[best_auc_idx]

print(f"Best AUC Configuration:")
print(f"  AUC: {best_auc_objectives[2]:.4f}")
print(f"  Sensitivity: {best_auc_objectives[0]:.4f}")
print(f"  Specificity: {best_auc_objectives[1]:.4f}")
print(f"\nConfiguration: {json.dumps(best_auc_config, indent=2)}")

# Save to file
with open(f"{OUTPUT_DIR}/best_auc_config.json", 'w') as f:
    json.dump({
        'config': best_auc_config,
        'objectives': {
            'sensitivity': float(best_auc_objectives[0]),
            'specificity': float(best_auc_objectives[1]),
            'auc': float(best_auc_objectives[2]),
            'model_size_M': float(best_auc_objectives[3]),
            'inference_time_ms': float(best_auc_objectives[4])
        }
    }, f, indent=2)

print(f"\nBest configuration saved to {OUTPUT_DIR}/best_auc_config.json")

## 12. Select Configuration by Preference (Optional)

Choose a configuration based on your priorities.

In [ ]:
# Find different optimal solutions
best_sensitivity_idx = np.argmax(pareto_F[:, 0])
best_specificity_idx = np.argmax(pareto_F[:, 1])
smallest_model_idx = np.argmin(pareto_F[:, 3])
fastest_inference_idx = np.argmin(pareto_F[:, 4])

print("Different Optimization Preferences:\n")

print(f"1. Best Sensitivity: {pareto_F[best_sensitivity_idx, 0]:.4f}")
print(f"   AUC: {pareto_F[best_sensitivity_idx, 2]:.4f}, Size: {pareto_F[best_sensitivity_idx, 3]:.2f}M\n")

print(f"2. Best Specificity: {pareto_F[best_specificity_idx, 1]:.4f}")
print(f"   AUC: {pareto_F[best_specificity_idx, 2]:.4f}, Size: {pareto_F[best_specificity_idx, 3]:.2f}M\n")

print(f"3. Best AUC: {pareto_F[best_auc_idx, 2]:.4f}")
print(f"   Sens: {pareto_F[best_auc_idx, 0]:.4f}, Spec: {pareto_F[best_auc_idx, 1]:.4f}\n")

print(f"4. Smallest Model: {pareto_F[smallest_model_idx, 3]:.2f}M")
print(f"   AUC: {pareto_F[smallest_model_idx, 2]:.4f}, Time: {pareto_F[smallest_model_idx, 4]:.2f}ms\n")

print(f"5. Fastest Inference: {pareto_F[fastest_inference_idx, 4]:.2f}ms")
print(f"   AUC: {pareto_F[fastest_inference_idx, 2]:.4f}, Size: {pareto_F[fastest_inference_idx, 3]:.2f}M")

## 13. Results Summary

All results have been saved to your Google Drive:
- Pareto-optimal configurations
- Optimization history
- Visualizations
- Best configuration JSON

You can now:
1. Download the results from Google Drive
2. Use the best configuration to train a final model
3. Deploy the model for inference
4. Run additional experiments with different parameters